In [ ]:
# Takes the output from routing_3.ipynb and works on the routing portion of it
import polars as pl
import numpy as np
import os
import rtsvg
rt = rtsvg.RACETrack()

In [ ]:
file_no = 13
df_collapsed    = pl.read_parquet(f'../../data/stanford/facebook/348.edges_collapsed_edges.{file_no}.parquet')
df_edge_arc_pos = pl.read_parquet(f'../../data/stanford/facebook/348.edges_arc_pos.{file_no}.parquet')
df_collapsed.sample(10)

In [ ]:
df_edge_arc_pos.sample(1)

In [ ]:
def extractCircles(_df_):
    circles, circle_lu = [], {}
    for k, k_df in _df_.group_by(['x_circle','y_circle','r_circle']):
        circle_lu[k] = len(circles)
        circles.append(k)
    return circles, circle_lu
def validatePositions(_df_, clearance=10): # edge_arc_pos
    # Extract the circles first
    circles, circle_lu = extractCircles(_df_)
    # Then make sure the positions don't fall within a circle & that they have some clearance
    for k, k_df in _df_.group_by(['x','y', 'x_circle', 'y_circle', 'r_circle']):
        _xy_     = (k[0], k[1])
        _uv_     = rt.unitVector((k[2:4], _xy_))
        _xy_end_ = (_xy_[0]+ clearance*_uv_[0], _xy_[1]+ clearance*_uv_[1])
        _my_circle_ = k[2:5]
        for c in circles:
            if c == _my_circle_: continue
            _length_ = rt.segmentLength((_xy_,c))
            if _length_ < c[2] + clearance:
                raise Exception(f'node key "{k_df["node_key"]}" with radius circle {c}')
            _length_ = rt.segmentLength((_xy_end_, c))

def extents(_df_): # edge_arc_pos -- assumes that the coordinates are already in screen space
    xmax, ymax = 10.0, 10.0
    for k, k_df in _df_.group_by(['x_circle','y_circle','r_circle']):
        xmax, ymax = max(xmax, k[0]+k[2]), max(ymax, k[1]+k[2])
    for k, k_df in _df_.group_by(['x','y']):
        xmax, ymax = max(xmax, k[0]), max(ymax, k[1])
    return (0.0, 0.0, xmax, ymax)

validatePositions(df_edge_arc_pos)
_ext_ = extents(df_edge_arc_pos)

In [ ]:
#
# isedgarVoronoiCircleEdition() - same as isedgarVoronoi() but consider circle radii when creating the bisectors
#
def isedgarVoronoiCircleEdition(self, S, Box=None, pad = 10):
    # return bisector of points p0 and p1 -- bisector is ((x,y),(u,v)) where u,v is the vector of the bisector
    # - this is the modified part
    def bisectorForCircles(c0, c1):
        uv              = self.unitVector((c0, c1))
        x0, y0          = c0[0] + uv[0] * c0[2], c0[1] + uv[1] * c0[2]
        x1, y1          = c1[0] - uv[0] * c1[2], c1[1] - uv[1] * c1[2]
        x,  y           = (x0+x1)/2.0, (y0+y1)/2.0
        pdx, pdy        = uv[1], -uv[0]
        return ((x,y), (pdx,pdy))
    def bisector(p0, p1):
        x, y     = (p0[0] + p1[0])/2.0, (p0[1] + p1[1])/2.0
        uv       = self.unitVector((p0, p1))
        pdx, pdy = uv[1], -uv[0]
        return ((x,y), (pdx,pdy))
    # returns vertices that intersect the polygon
    def intersects(bisects, poly):
        inters = []
        for i in range(0, len(poly)):
            p0, p1 = poly[i], poly[(i+1)%len(poly)]
            xy = self.lineSegmentIntersectionPoint((bisects[0], (bisects[0][0] + bisects[1][0], bisects[0][1] + bisects[1][1])),
                                                    (p0, p1))
            if xy is not None and xy != p1: inters.append((xy, i, (i+1)%len(poly)))
        return inters
    # conctenate a point, a list, and a point into a new list
    def createCell(my_cell, p0, p1, i0, i1, sign):
        l = [p0]
        i = i0
        while i != i1:
            l.append(my_cell[i])
            i += sign
            if i < 0:             i += len(my_cell)
            if i >= len(my_cell): i -= len(my_cell)
        l.append(my_cell[i1])
        l.append(p1)
        return l
    # contain true if pt is in poly
    def contains(poly, pt):
        inter_count = 0
        for i in range(len(poly)):
            p0, p1 = poly[i], poly[(i+1)%len(poly)]
            _tuple_ = self.segmentsIntersect((pt,(pt[0]+1e9,pt[1])),(p0,p1)) # use a ray from the pt to test
            if _tuple_[0] and (_tuple_[1], _tuple_[2]) != p1: inter_count += 1
        if inter_count%2 == 1: return True
        return False
    # actual algorithm
    if Box is None:
        x_l, x_r, y_t, y_b = S[0][0], S[0][0], S[0][1], S[0][1]
        for pt in S: x_l, x_r, y_t, y_b = min(x_l, pt[0]), max(x_r, pt[0]), max(y_t, pt[1]), min(y_b, pt[1])
        Box = [(x_l-pad, y_t+pad), (x_r+pad, y_t+pad), (x_r+pad, y_b-pad), (x_l-pad, y_b-pad)]
    cells = []
    for i in range(len(S)):
        p    = S[i]
        cell = Box
        for q in S:
            if p == q: continue
            B            = bisectorForCircles(p,q)
            B_intersects = intersects(B, cell)
            if len(B_intersects) == 2:
                t1, t2 = B_intersects[0][0], B_intersects[1][0]
                xi, xj = B_intersects[0][2], B_intersects[1][1]
                newCell      = createCell(cell, t1, t2, xi, xj, 1)
                xi, xj = B_intersects[0][1], B_intersects[1][2]
                otherNewCell = createCell(cell, t1, t2, xi, xj, -1)
                if contains(newCell, p) == False: newCell = otherNewCell
                cell = newCell
        cells.append(cell)
    return cells

x_ins, y_ins = 50, 50
# Base Circles
svg_base = [f'<svg x="0" y="0" width="{_ext_[2]+x_ins}" height="{_ext_[3]+y_ins}">']
svg_base.append(f'<rect x="0" y="0" width="{_ext_[2]+x_ins}" height="{_ext_[3]+y_ins}" fill="#ffffff" />')
circles, circles_lu = extractCircles(df_edge_arc_pos)
for i in range(len(circles)):
    cx, cy, r = circles[i]
    svg_base.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="3" />')
# Entry / Exit Points
svg_pts = []
for k, k_df in df_edge_arc_pos.group_by(['node_key', 'x', 'y', 'x_circle', 'y_circle', 'r_circle']):
    if '_to_' in k[0]: _color_, _fill_ = '#ff0000', 'none'
    else:              _color_, _fill_ = '#00ff00', '#00b000'
    _xy_, _cxy_ = k[1:3], k[3:5]
    svg_pts.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="4" fill="{_fill_}" stroke="{_color_}" stroke-width="0.5"/>')
    _uv_        = rt.unitVector((_cxy_, _xy_))
    svg_pts.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{_xy_[0]+10*_uv_[0]}" y2="{_xy_[1]+10*_uv_[1]}" stroke="#404040" stroke-width="1.0"/>')
# Delauney Triangulations
_box_ = [(_ext_[0],_ext_[1]),(_ext_[0],_ext_[3]+y_ins),(_ext_[2]+x_ins,_ext_[3]+y_ins),(_ext_[2]+x_ins,_ext_[1])]
voronoi_polys = rt.isedgarVoronoi(circles, Box=_box_, use_circle_radius=True)
svg_voronoi = []
for i in range(len(voronoi_polys)):
    d = f'M {voronoi_polys[i][0][0]} {voronoi_polys[i][0][1]} '
    for j in range(1, len(voronoi_polys[i])): d += f'L {voronoi_polys[i][j][0]} {voronoi_polys[i][j][1]} '
    d += 'Z'
    svg_voronoi.append(f'<path d="{d}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="10"/>')

rt.tile([''.join(svg_base)+''.join(svg_pts)+''.join(svg_voronoi)+'</svg>'])